In [ ]:
import helper as h
import functions as f
import graph_functions as gf

In [ ]:
h.df.groupby('airline')['class'].unique()

In [ ]:
#Flights with no stops
df_nonstop = h.df[(h.df['class'] == 'Economy') & (h.df['stops'] == '0')]

In [ ]:
#Flights with 1 stop
df_1stop = h.df[(h.df['class'] == 'Economy') & (h.df['stops'] == '1')]

In [ ]:
#Flights with 2 or more stops
df_2more_stop = h.df[(h.df['class'] == 'Economy') & (h.df['stops'] == '2+')]

In [ ]:
gf.price_airline(df_nonstop)

In [ ]:
outliers = df_nonstop.groupby('airline', group_keys=False).apply(gf.detect_outliers_iqr)
print(f"Total outliers detected: {len(outliers)}")

In [ ]:
print(f"Outlier percentage: {len(outliers) / len(df_nonstop):.2%}")

In [ ]:
for airline, group in df.groupby('airline'):
    stat, p_value = h.shapiro(group['price'])
    print(f"{airline}: W = {stat:.4f}, p-value = {p_value:.4f}")

In [ ]:
from scipy.stats import f_oneway

groups = [group["price"].values for name, group in df.groupby("airline")]

# Perform one-way ANOVA
f_stat, p_value = f_oneway(*groups)

print("F-statistic:", f_stat)
print("p-value:", p_value)

In [ ]:
import scipy.stats as st

summary_df = df_nonstop.groupby('airline').agg({"price": ['mean', 'std', 'count','sem']}).reset_index()
summary_df.columns = ['airline', 'mean', 'std', 'count','sem'] 
summary_df['lower_bound'] = summary_df['mean'] - (st.norm.ppf(1-(0.05/2))*summary_df['sem'])
summary_df['upper_bound'] = summary_df['mean'] + (st.norm.ppf(1-(0.05/2))*summary_df['sem'])
summary_df['error'] = (st.norm.ppf(1-(0.05/2))*summary_df['sem'])
summary_df.sort_values('mean', ascending=False, inplace=True)
display(summary_df)

palette = sns.color_palette("Set3", len(summary_df))
plt.errorbar(summary_df['airline'], summary_df['mean'], yerr=summary_df['error'], fmt='o', 
             color='black', elinewidth=3, capthick=3, errorevery=1, alpha=1, ms=4, capsize=5)
plt.bar(summary_df['airline'], summary_df['mean'], color=palette, tick_label=summary_df['airline'])
plt.xlabel("Airline")
plt.ylabel("Mean Delay Log + CI")
plt.yscale('log')
plt.show()

In [ ]:
# Compute mean price per airline
mean_prices = df_nonstop.groupby('airline')['price'].mean().reset_index()

# Sort by mean price (optional, for readability)
mean_prices = mean_prices.sort_values('price', ascending=False)

# Plot
plt.figure(figsize=(10,6))
sns.barplot(data=mean_prices, x='airline', y='price', palette='viridis')

plt.title('Mean Ticket Price by Airline')
plt.xlabel('Airline')
plt.ylabel('Mean Price')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
gf.price_airline(df_1stop)

In [ ]:
outliers = df_1stop.groupby('airline', group_keys=False).apply(gf.detect_outliers_iqr)
print(f"Total outliers detected: {len(outliers)}")

In [ ]:
print(f"Outlier percentage: {len(outliers) / len(df_nonstop):.2%}")

In [ ]:
groups = [df_1stop[df_1stop['airline'] == airline]['price'] for airline in df_1stop['airline'].unique()]

stat, p = h.kruskal(*groups)

print("Kruskal–Wallis H-statistic:", stat)
print("p-value:", p)

In [ ]:
# Compute mean price per airline
mean_prices = df_1stop.groupby('airline')['price'].mean().reset_index()

# Sort by mean price (optional, for readability)
mean_prices = mean_prices.sort_values('price', ascending=False)

# Plot
plt.figure(figsize=(10,6))
sns.barplot(data=mean_prices, x='airline', y='price', palette='viridis')

plt.title('Mean Ticket Price by Airline')
plt.xlabel('Airline')
plt.ylabel('Mean Price')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import scipy.stats as st

summary_df = df_1stop.groupby('airline').agg({"price": ['mean', 'std', 'count','sem']}).reset_index()
summary_df.columns = ['airline', 'mean', 'std', 'count','sem'] 
summary_df['lower_bound'] = summary_df['mean'] - (st.norm.ppf(1-(0.05/2))*summary_df['sem'])
summary_df['upper_bound'] = summary_df['mean'] + (st.norm.ppf(1-(0.05/2))*summary_df['sem'])
summary_df['error'] = (st.norm.ppf(1-(0.05/2))*summary_df['sem'])
summary_df.sort_values('mean', ascending=False, inplace=True)
display(summary_df)

plt.errorbar(summary_df['airline'], summary_df['mean'], yerr=summary_df['error'], fmt='o', 
             color='black', elinewidth=3, capthick=3, errorevery=1, alpha=1, ms=4, capsize=5)
plt.bar(summary_df['airline'], summary_df['mean'], tick_label=summary_df['airline'])
plt.xlabel("airline")
plt.ylabel("mean delay")
plt.yscale('log')
plt.show()

In [ ]:
gf.price_airline(df_2more_stop)

In [ ]:
outliers = df_2more_stop.groupby('airline', group_keys=False).apply(gf.detect_outliers_iqr)
print(f"Total outliers detected: {len(outliers)}")

In [ ]:
print(f"Outlier percentage: {len(outliers) / len(df_nonstop):.2%}")

In [ ]:
groups = [df_2more_stop[df_2more_stop['airline'] == airline]['price'] for airline in df_2more_stop['airline'].unique()]

stat, p = hm.kruskal(*groups)

print("Kruskal–Wallis H-statistic:", stat)
print("p-value:", p)

In [ ]:
# Compute mean price per airline
mean_prices = df_2more_stop.groupby('airline')['price'].mean().reset_index()

# Sort by mean price (optional, for readability)
mean_prices = mean_prices.sort_values('price', ascending=False)

# Plot
plt.figure(figsize=(10,6))
sns.barplot(data=mean_prices, x='airline', y='price', palette='viridis')

plt.title('Mean Ticket Price by Airline')
plt.xlabel('Airline')
plt.ylabel('Mean Price')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# total rows
total_rows = len(df_nonstop)

# number of rows where lead_time_days < 15
last_minute = len(df_nonstop[df_nonstop['lead_time_days'] < 15])

# percentage
percentage_last_minute = (last_minute / total_rows) * 100

print(f"Percentage of bookings made less than 15 days before departure: {percentage_last_minute:.2f}%")

In [ ]:
from scipy.stats import spearmanr

# Drop NaN or invalid values
df_valid = df_nonstop[['lead_time_days', 'price']].dropna()

# Spearman correlation
corr, pval = spearmanr(df_valid['lead_time_days'], df_valid['price'])

print(f"Spearman correlation = {corr:.3f}")
print(f"p-value = {pval:.5f}")

if pval < 0.05:
    print("Reject H0 → There is a statistically significant relationship between lead time and price.")
else:
    print("Fail to reject H0 → No statistically significant relationship found.")

In [ ]:
plt.figure(figsize=(10,6))
sns.lmplot(
    data=df_nonstop,
    x='lead_time_days',
    y='price',
    hue='airline',
    lowess=True,
    height=6,
    aspect=1.5,
    scatter=False,
    palette='husl'
)

plt.title('Average Price Trend by Lead Time')
plt.xlabel('Lead Time (days)')
plt.ylabel('Ticket Price')
plt.tight_layout()
plt.show()

In [ ]:
# Convert duration ("HH:MM:SS") → total minutes
h.df['duration'] = (
    pd.to_timedelta(h.df['duration']).dt.total_seconds() / 60
)

# Keep relevant columns
h.df = h.df[['airline', 'duration', 'price']]

# Remove invalid rows (zero or null durations/prices)
h.df = h.df[(h.df['duration'] > 0) & (h.df['price'] > 0)]

In [ ]:
plt.figure(figsize=(8,6))
plt.hexbin(h.df['duration'], h.df['price'], gridsize=50, cmap='viridis', bins='log')
plt.colorbar(label='log10(Number of flights)')
plt.title("Price vs Duration")
plt.xlabel("Duration (minutes)")
plt.ylabel("Price (INR)")
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import pearsonr, spearmanr

# Pearson correlation (linear)
pearson_r, pearson_p = pearsonr(h.df['duration'], h.df['price'])
# Spearman correlation (monotonic)
spearman_r, spearman_p = spearmanr(h.df['duration'], h.df['price'])

print(f"Pearson r = {pearson_r:.3f} (p = {pearson_p:.3e})")
print(f"Spearman r = {spearman_r:.3f} (p = {spearman_p:.3e})")

In [ ]:
df = pd.read_csv(config['data']['clean_data']['full_clean'], sep=";")

In [ ]:
df['stops'] = df['stops'].replace('2+', '2')

In [ ]:
# Keep relevant columns
df_stops = df[['stops', 'price']].copy()

# Ensure 'stops' is numeric
df_stops['stops'] = pd.to_numeric(df_stops['stops'], errors='coerce')

# Drop missing or invalid values
df_stops = df_stops.dropna(subset=['stops', 'price'])

In [ ]:
plt.figure(figsize=(7,5))
sns.boxplot(x='stops', y='price', data=df_stops)
plt.title("Ticket Prices by Number of Stops")
plt.xlabel("Number of Stops")
plt.ylabel("Price")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
for stops in sorted(df_stops['stops'].unique()):
    subset = df_stops[df_stops['stops'] == stops]
    sns.kdeplot(subset['price'], label=f'{int(stops)} stop(s)', fill=True)

plt.title("Price Density by Number of Stops")
plt.xlabel("Ticket Price")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
df_stops['flight_type'] = df_stops['stops'].apply(lambda x: 'Direct' if x == 0 else 'Connecting')


In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(x='flight_type', y='price', data=df_stops, ci=95, capsize=0.2)
plt.title("Average Ticket Price by Number of Stops (95% CI)")
plt.xlabel("Number of Stops")
plt.ylabel("Average Price")
plt.tight_layout()
plt.show()

In [ ]:
price_by_stops = df_stops.groupby('stops')['price'].describe()[['mean','std','count']]
print(price_by_stops)

In [ ]:
# Create the 'flight_type' variable
df_stops['flight_type'] = df_stops['stops'].apply(lambda x: 'Direct' if x == 0 else 'Connecting')

# Split prices into two groups
direct_prices = df_stops[df_stops['flight_type'] == 'Direct']['price']
connecting_prices = df_stops[df_stops['flight_type'] == 'Connecting']['price']

# Run one-way ANOVA
f_stat, p_value = stats.f_oneway(direct_prices, connecting_prices)

print(f"ANOVA F-statistic: {f_stat:.3f}, p-value: {p_value:.3e}")

In [ ]:
df_class = df[['class', 'price']].copy()
df_class['class'] = df_class['class'].str.strip().str.capitalize()
df_class = df_class.dropna(subset=['class', 'price'])

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='class', y='price', data=df_class)
plt.title("Ticket Price by Travel Class")
plt.xlabel("Class")
plt.ylabel("Price (INR)")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(x='class', y='price', data=df, ci=95, palette='pastel', capsize=0.2)
plt.title("Average Price by Class (with 95% CI)")
plt.xlabel("Travel Class")
plt.ylabel("Mean Price (INR)")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.kdeplot(df[df['class'] == 'Economy']['price'], label='Economy', fill=True)
sns.kdeplot(df[df['class'] == 'Business']['price'], label='Business', fill=True)
plt.title("Distribution of Ticket Prices by Class")
plt.xlabel("Price (INR)")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
price_by_class = df_class.groupby('class')['price'].describe()[['mean','std','count']]
print(price_by_class)

In [ ]:
economy = df_class.loc[df_class['class'] == 'Economy', 'price']
business = df_class.loc[df_class['class'] == 'Business', 'price']

t_stat, p_value = stats.ttest_ind(economy, business, equal_var=False)
print(f"T-statistic: {t_stat:.3f}, p-value: {p_value:.3e}")